<a href="https://colab.research.google.com/github/operationsgrupovoga/AnalisePredicaoReceita/blob/main/Script_predicao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análise Preditiva de Receita:**
A ideia desta análise foi fazer um modelo para prever a receita mensal do escritório num horizonte de 6 meses, foram levados em conta:

- NNM e 3 lags de NNM (3 meses para trás)
- log do AuC (Foi feita transformação com escala logarítmica para estabilização)
- Giro
- Clientes ativos
- Trasações
- Dias úteis

> O modelo deve ser rodado mês a mês a medida que novos dados se tornam disponíveis.

Foram feitos 2 testes estátisticos para séries temporais:

- ### ARIMA (Autoregressive Integrated Moving Average)

- ### XREG

#### As métricas resultantes do teste são:

* MAPE: Erro médio em termos percentuais (50% ruim, < 10% excelente)

* DirAcc (Directional Accuracy): acurácia em prever se a receita vai subir ou cair (ex: de 83% = acerta a direção em 5 de 6 meses)

* MAE: Erro médio em unidades monetárias


### Acompanhamento de resultados:

**Setembro:**
- Esperado: R$ 724.140,00

- Real: R$ 751.861,3 (16/10/2025)

**Outubro:**
- Esperado: R$ 1.000.135,00


## Requisitos:
### Bibliotecas e pacotes que serão utilizados:
- readxl      # Leitura de arquivos Excel
- tidyverse   # Manipulação de dados
- lubridate   # Manipulação de datas
- janitor     # Limpeza de nomes de colunas
- dplyr       # Transformação de dados
- forecast    # Modelos de séries temporais
- timetk      # Divisão temporal de dados
- rsample     # Amostragem e split de dados

In [ ]:
# =========================================================

# 0) Pacotes e bibliotecas

# =========================================================
library(readxl)
library(tidyverse)
library(lubridate)
library(dplyr)


pkgs <- c("forecast","janitor","caret","timetk","rsample")

to_install <- pkgs[!pkgs %in% installed.packages()[,"Package"]]

if (length(to_install)) install.packages(to_install, quiet = TRUE)

invisible(lapply(pkgs, library, character.only = TRUE))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.1     ✔ stringr   1.5.2
✔ ggplot2   4.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
also installing the dependencies ‘shape’, ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘TTR’, ‘prodlim’, ‘listenv’, ‘parallelly’, ‘quadprog’, ‘quantmod’, ‘proxy’, ‘iterators’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘sparsevctrs’, ‘lazyeval’, ‘crosstalk’, ‘warp’, ‘BH’, ‘RcppRoll’, ‘future’, ‘globals’, ‘colorspace’, ‘fracdiff’, ‘lmtest’, ‘timeDate’, ‘tseries’, ‘urca’, ‘zoo’, ‘RcppArmadillo’, ‘snakecase’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘

### Carregamento + tratamento dos dados + padronização de nomes/ acentos/ formatos (data) e ordenação


Formato: Excel (.xlsx)


Estrutura: Dados mensais com os campos (mes, receita, nm, auc, clientes_ativos, transacoes, giro, dias_uteis)

In [ ]:
#Arquivo:
receitas <- read_excel("AnáliseReceita_Asdra.xlsx",
                       sheet = "Mensal")

receitas <- receitas %>% select(-volume)
# =========================================================

# 1) Base 'receitas' (sem volume) -> limpeza

#    colunas mín.: mes, receita, nnm, auc

#    opcionais: clientes_ativos, transacoes, giro, dias_uteis

# =========================================================

stopifnot(exists("receitas"))

dados <- receitas %>%

  as_tibble() %>%

  janitor::clean_names() %>%      # "mês"->mes; "transações"->transacoes; "dias úteis"->dias_uteis

  mutate(

    mes = as.Date(mes),

    # garantir tipos numéricos

    across(c(nnm, auc, receita, clientes_ativos, transacoes, giro, dias_uteis),

           ~ suppressWarnings(as.numeric(.x)))

  ) %>%

  arrange(mes)


Geração de features (lags e transformações)
- 3 defasagens de NNM (1,2 e 3 meses atrás) > lags
- Receita do mês anterior (lag de 1 mês da receita) > receita_l1
- Linearização do AuC para normalização (estabilizar a variância dos dados e reduzir o impacto de outliers) > auc_log = transformação logarítmica do auc
- Remoção de linhas NaN após os lags


### Split temporal treino/teste

Definição de dados de treino como todos os dados até os últimos 6 meses.

Definição dos dados de teste como os últimos 6 meses.

Neste caso temos 30 meses sendo analisados >> 24 foram para treino, e 6 para teste.


In [ ]:

# =========================================================

# 2) Features (lags SEM vazamento)

# =========================================================

df_feat <- dados %>%

  arrange(mes) %>%

  mutate(

    nnm_l1 = lag(nnm, 1),

    nnm_l2 = lag(nnm, 2),

    nnm_l3 = lag(nnm, 3),

    receita_l1 = lag(receita, 1),

    auc_log    = log1p(auc)

  ) %>%

  tidyr::drop_na(nnm_l1, nnm_l2, nnm_l3, receita_l1)


# =========================================================

# 3) Split temporal (últimos 6 meses de teste)

# =========================================================

split <- timetk::time_series_split(df_feat, assess = 6, cumulative = TRUE, date_var = mes)

train_data <- training(split)

test_data  <- testing(split)



## Preparação das variáveis para teste XREG

Com intuito de garantir um formato confiável para X, evitar problemas de multicolinearidade/escala e garantir reprodutibilidade.


- Define um conjunto de candidatas xreg_cand (nnm, lags, auc_log, giro, clientes_ativos, transacoes, dias_uteis)

- Remove colinearidade muito alta (acima de 0.99)

- Garante posto completo (garante que não haverão colunas linearmente depedentes), se ainda houver dependência linear, escolhe subconjunto independente

- Padroniza por z score

- Preenchimento de valores faltantes do conjunto de teste com 0






In [ ]:

# =========================================================

# 4) Helpers p/ XREG (limpa -> full-rank -> escala e guarda template)

# =========================================================

build_xreg_template <- function(df, wanted, cor_cut = 0.999) {

  X <- df[, intersect(wanted, names(df)), drop = FALSE]

  stopifnot(ncol(X) > 0)

  X <- dplyr::mutate(X, dplyr::across(dplyr::everything(), as.numeric))

  # remove NA/Inf

  good <- sapply(X, function(v) all(is.finite(v)))

  X <- X[, good, drop = FALSE]; stopifnot(ncol(X) > 0)

  # remove variância ~0

  nzv <- sapply(X, function(v) sd(v, na.rm = TRUE) > 0)

  X <- X[, nzv, drop = FALSE]; stopifnot(ncol(X) > 0)

  # remove colinearidade alta

  if (ncol(X) > 1){

    keep <- rep(TRUE, ncol(X))

    for (i in seq_len(ncol(X))) {

      if (!keep[i]) next

      cor_i  <- abs(cor(X[, i], X[, keep, drop = FALSE], use = "pairwise.complete.obs"))

      drop_j <- which(keep)[which(cor_i >= cor_cut)]

      drop_j <- drop_j[drop_j > i]

      if (length(drop_j)) keep[drop_j] <- FALSE

    }

    X <- X[, keep, drop = FALSE]

  }

  # full-rank

  if (ncol(X) > 1){

    q <- qr(as.matrix(X))

    if (q$rank < ncol(X)) X <- X[, q$pivot[seq_len(q$rank)], drop = FALSE]

  }

  # escala e guarda

  Xs <- scale(as.matrix(X))

  list(X = Xs, cols = colnames(X),

       center = attr(Xs, "scaled:center"),

       scale  = attr(Xs, "scaled:scale"))

}

transform_xreg <- function(df, template) {

  X <- df[, intersect(template$cols, names(df)), drop = FALSE]

  miss <- setdiff(template$cols, names(X))

  if (length(miss)) X[miss] <- 0

  X <- X[, template$cols, drop = FALSE]

  X <- dplyr::mutate(X, dplyr::across(dplyr::everything(), as.numeric)) |> as.matrix()

  scale(X, center = template$center, scale = template$scale)

}

### Rodando os modelos:


In [ ]:

# =========================================================

# 5) XREG de treino/teste

# =========================================================

xreg_cand <- c("nnm","nnm_l1","nnm_l2","nnm_l3",

               "auc_log","giro","clientes_ativos","transacoes","dias_uteis")

xreg_cand <- intersect(xreg_cand, names(train_data))

templ      <- build_xreg_template(train_data, xreg_cand)

xreg_train <- templ$X; X_train_df <- as.data.frame(xreg_train)

y_train    <- train_data$receita

y_train_log <- log1p(y_train)

y_ts_log    <- ts(y_train_log, frequency = 12)

xreg_test <- transform_xreg(test_data, templ)

X_test_df <- as.data.frame(xreg_test)

y_test    <- test_data$receita

# =========================================================

# 6) Modelos em LOG: ARIMAX + TSLM

# =========================================================

fit_arimax_log <- try(

  forecast::auto.arima(

    y_ts_log, xreg = xreg_train,

    seasonal = TRUE, stepwise = TRUE, approximation = TRUE,

    allowmean = FALSE, allowdrift = FALSE

  ), silent = TRUE

)

if (inherits(fit_arimax_log, "try-error")) {

  fit_arimax_log <- forecast::auto.arima(

    y_ts_log, xreg = xreg_train,

    seasonal = FALSE, stepwise = FALSE, approximation = TRUE,

    allowmean = FALSE, allowdrift = FALSE

  )

}

fit_tslm_log <- forecast::tslm(y_ts_log ~ trend + season + ., data = X_train_df)


### Avaliação do resultado dos testes

MAPE: erro percentual médio

MAE: erro absoluto médio (em unidades de receita)

DirAcc: acurácia direcional (% de acertos na direção da mudança)

In [ ]:
# =========================================================

# 7) Avaliação no TESTE e pesos do ensemble (últimos 12 meses do treino)

# =========================================================

pred_log_arima_test <- as.numeric(forecast::forecast(fit_arimax_log, xreg = xreg_test, h = nrow(test_data))$mean)

pred_log_tslm_test  <- as.numeric(forecast::forecast(fit_tslm_log, newdata = X_test_df)$mean)

# pesos por MAPE na janela recente do treino

k <- min(12, nrow(X_train_df)); idx <- (nrow(X_train_df)-k+1):nrow(X_train_df)

pred_log_arima_val <- tryCatch(as.numeric(fitted(fit_arimax_log))[idx], error=function(e) rep(NA_real_, k))

pred_log_tslm_val  <- tryCatch(as.numeric(predict(fit_tslm_log, newdata=X_train_df))[idx], error=function(e) rep(NA_real_, k))

y_val_log <- y_train_log[idx]

mape_fn <- function(y_t, y_p){ mean(abs(expm1(y_t)-expm1(y_p))/pmax(expm1(y_t),1), na.rm=TRUE) }

w_arima <- (1/mape_fn(y_val_log, pred_log_arima_val)); w_tslm <- (1/mape_fn(y_val_log, pred_log_tslm_val))

ws <- w_arima + w_tslm; w_arima <- w_arima/ws; w_tslm <- w_tslm/ws

Plota reais vs. previstos (linha sólida = real, pontos/linha tracejada = previsto)

In [ ]:
pred_test_log_ens <- w_arima*pred_log_arima_test + w_tslm*pred_log_tslm_test

pred_test <- expm1(pred_test_log_ens)

mape <- mean(abs((y_test - pred_test)/pmax(y_test,1))); mae <- mean(abs(y_test - pred_test))

dir_acc <- mean(sign(diff(pred_test)) == sign(diff(y_test)))

cat("\nTESTE -> MAPE:", round(mape,3), "| MAE:", round(mae,2), "| DirAcc:", round(dir_acc,2), "\n")

plot(test_data$mes, y_test, type="l", lwd=2, xlab="mês", ylab="receita",

     main="Receita: Real (linha) x Prevista (pontos)")

points(test_data$mes, pred_test, pch=19); lines(test_data$mes, pred_test, lty=2)


Projeção de cenários futuros (6 meses à frente)

Constrói cenários simples para as xregs futuras: (alteramos aqui de acordo com as nossas expectativas)
  - nnm: média dos últimos 6 meses
  - auc: cresce 0,5% a.m a partir do último valor
  - clientes_ativos, transacoes: mantém último valor (persistência)
  - giro: mediana dos últimos 6
  - dias_uteis: mediana dos últimos 12; fallback 21.

Geração dos lags futuros de nnm de forma recursiva (sem NA):
- usa os 3 últimos históricos para preencher os primeiros passos
- depois usa os próprios valores projetados (nnm_scn) defasados

Mês 1: Usa histórico real

Mês 2: Usa histórico + previsão do mês 1

Mês 3+: Usa previsões anteriores

In [ ]:

# =========================================================

# 8) FORECAST FUTURO (próximos h meses) – sem volume

#    Constrói lags futuros de NNM sem NA e cria cenários simples.

#    >>> Ajuste os vetores *_scn conforme sua realidade.

# =========================================================

h <- 6  # horizonte desejado

# --- 8.1 cenários simples (ajuste livre) ---

nnm_scn <- rep(mean(tail(dados$nnm, 6), na.rm = TRUE), h)         # NNM ~ média recente

auc_scn <- tail(dados$auc, 1) * cumprod(1 + rep(0.005, h))        # AUC +0,5% a.m.

cli_scn <- if ("clientes_ativos" %in% names(dados)) rep(tail(dados$clientes_ativos,1), h) else rep(0, h)

trx_scn <- if ("transacoes"      %in% names(dados)) rep(tail(dados$transacoes,1),      h) else rep(0, h)

giro_scn<- if ("giro"            %in% names(dados)) rep(median(tail(dados$giro,6),na.rm=TRUE), h) else rep(0, h)

dias_scn<- if ("dias_uteis"      %in% names(dados)) rep(median(tail(dados$dias_uteis,12),na.rm=TRUE), h) else rep(21L, h)

# --- 8.2 lags de NNM para o futuro (recursivo, sem NA) ---

nnm_hist <- tail(df_feat$nnm, 3)

nnm_l1_fut <- nnm_l2_fut <- nnm_l3_fut <- numeric(h)

for (t in 1:h) {

  nnm_l1_fut[t] <- if (t == 1) nnm_hist[3] else nnm_scn[t-1]

  nnm_l2_fut[t] <- if (t == 1) nnm_hist[2] else if (t == 2) nnm_hist[3] else nnm_scn[t-2]

  nnm_l3_fut[t] <- if (t == 1) nnm_hist[1] else if (t == 2) nnm_hist[2] else if (t == 3) nnm_hist[3] else nnm_scn[t-3]

}

# --- 8.3 monta future_df e transforma com o MESMO template ---

future_df <- tibble(

  mes = seq(max(dados$mes) %m+% months(1), by = "1 month", length.out = h),

  nnm = nnm_scn,

  auc = auc_scn,

  clientes_ativos = cli_scn,

  transacoes = trx_scn,

  giro = giro_scn,

  dias_uteis = dias_scn

) %>%

  mutate(

    auc_log = log1p(auc),

    nnm_l1  = nnm_l1_fut,

    nnm_l2  = nnm_l2_fut,

    nnm_l3  = nnm_l3_fut

  )

xreg_future <- transform_xreg(future_df, templ)

X_future_df <- as.data.frame(xreg_future)

# --- 8.4 previsão (LOG) por modelo + ensemble com pesos do treino ---

pred_log_arima_fut <- as.numeric(forecast::forecast(fit_arimax_log, xreg = xreg_future, h = nrow(X_future_df))$mean)

pred_log_tslm_fut  <- as.numeric(forecast::forecast(fit_tslm_log, newdata = X_future_df)$mean)

pred_log_fut_ens   <- w_arima*pred_log_arima_fut + w_tslm*pred_log_tslm_fut

pred_fut           <- pmax(expm1(pred_log_fut_ens), 0)

previsoes <- tibble(mes = future_df$mes, receita_prevista = pred_fut)

print(previsoes)

plot(previsoes$mes, previsoes$receita_prevista, type="l", lwd=2,

     xlab="mês", ylab="receita prevista", main="Previsão 6 meses (ensemble log)")
